Estudo baseado nos seguintes artigos:

* [Having an Imbalanced Dataset? Here Is How You Can Fix It.](https://towardsdatascience.com/having-an-imbalanced-dataset-here-is-how-you-can-solve-it-1640568947eb)

O dataset usado pode ser baixado [aqui](https://www.kaggle.com/mlg-ulb/creditcardfraud/version/3).